In [1]:
'''
author: Damiano Pasquini
date: 10/13/2023
'''

import torch
import subprocess
import json
import pandas as pd
import numpy
import transformers
import datasets
import tiktoken
import wandb
import tqdm

# Check if CUDA (GPU support) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor_on_device = torch.randn(3, 3).to(device)
print("Torch version:", torch.__version__)
print("Is CUDA enabled?", torch.cuda.is_available())

corpus = "nanoGPT/data/utterances.jsonl"
corpus_preprocessed = "nanoGPT/data/shakespeare_char/movies_text_preprocessed.txt"
# cli commands
prepare = "python nanoGPT/data/shakespeare_char/prepare.py"
sample = "python nanoGPT/sample.py --out_dir=out-shakespeare-char --device=cpu"

Torch version: 2.1.0+cu121
Is CUDA enabled? True


In [99]:
# from convokit import Corpus, download
# corpus = Corpus(filename=download("supreme-corpus")) 

In [105]:
def count_words(input=corpus):
    words = subprocess.check_output(f"wsl wc -w {input}", shell=True)
    return int(words.split()[0])

print(count_words(corpus_preprocessed))

1000029


In [2]:
# run a shell command and print the output
def run_shell_cmd(cmd):
    print(cmd)
    out = subprocess.check_output(cmd, shell=True)
    print(out)

In [3]:
def preprocess_corpus(input=corpus, output=corpus_preprocessed):
    # transform the jsonl file into a txt file, with the format "speaker: text". When the output file contains more than 1000000 words, stop. Divide the jsonl file into lines basing on the presence of "id":
    with open(input, "r") as f:
        text = f.read()
    word_counter = 0
    with open(output, "w", encoding='utf-8') as f:
        for line in text.replace("\\n", ". ").split("\n"):
            if word_counter > 1000000:
                break
            if line:
                line = json.loads(line)
                word_counter += len(line["text"].split())+len(line["speaker"].split())
                f.write(f"{line['speaker']}: {line['text']}\n")
    return output

preprocess_corpus(corpus)

'nanoGPT/data/shakespeare_char/utterances_preprocessed.txt'

In [4]:
# default training
def train(eval_iters=20, log_interval=1, block_size=64, batch_size=12, n_layer=4, n_head=4, n_embd=128, max_iters=2000, lr_decay_iters=2000, dropout=0.0):
    run_shell_cmd(f"python nanoGPT/train.py nanoGPT/config/train_shakespeare_char.py --device=cpu --compile=False --eval_iters={eval_iters} --log_interval={log_interval} --block_size={block_size} --batch_size={batch_size} --n_layer={n_layer} --n_head={n_head} --n_embd={n_embd} --max_iters={max_iters} --lr_decay_iters={lr_decay_iters} --dropout={dropout}")

# Prepare the data, train and sample from the model with the default parameters

In [111]:
# training with default hyperparameters
print("Running prepare...")
run_shell_cmd(prepare)
print("Training...")
train()
print("Sampling...")
run_shell_cmd(sample)

Running prepare...
python nanoGPT/data/shakespeare_char/prepare.py
b'length of dataset in characters: 5,830,837\r\nall the unique characters: \r\n "$%&\'()+,-./0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz\xe7\xe9\xef\x96\x91\x93\x94\r\nvocab size: 91\r\ntrain has 5,247,753 tokens\r\nval has 583,084 tokens\r\n'
Training...
python nanoGPT/train.py nanoGPT/config/train_shakespeare_char.py --device=cpu --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0


CalledProcessError: Command 'python nanoGPT/train.py nanoGPT/config/train_shakespeare_char.py --device=cpu --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0' returned non-zero exit status 1.

# Prepare the data, train and sample from the model with different configurations of hyperparameters

In [ ]:
# train 1
train()
print("Sampling...")
run_shell_cmd(sample)

In [ ]:
# train 2
train()
print("Sampling...")
run_shell_cmd(sample)

In [ ]:
# train 3
train()
print("Sampling...")
run_shell_cmd(sample)

KeyboardInterrupt: 